<a href="https://colab.research.google.com/github/cccaaannn/deep_learning_colab/blob/master/LSTM_vs_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

questions

The plots and results that will be generated in this homework:

1) Plot of the original dataset.

2) Plot of mean squared error of LSTM-RNN with changing timestep on test set.

3) Plot of mean squared error of MLP with changing timestep on test set.

4) Plot of the predictions of best MLP and LSTM-RNNs models along with the actual values of training set.

5) Plot of the predictions of best MLP and LSTM-RNNs models along with the actual values of test set.

6) Table including the Mean Squared Error and Mean Absolute error for both training and test sets.

imports


In [0]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, LSTM, Activation
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

load data

In [0]:
df = pd.read_csv('data.csv', index_col=0, sep = ',')
df

plot dataframe

In [0]:
plt.plot(df)
plt.show()

better plot

In [0]:
import matplotlib.dates as mdates
fig, ax = plt.subplots()
fig.autofmt_xdate()
plt.xticks(np.arange(0, 3000, 200))
ax.fmt_xdata = mdates.DateFormatter('%Y-%m-%d')
ax.plot(df)

split train test

In [0]:
mydataset = df.values
myscaler = StandardScaler()
mydataset = myscaler.fit_transform(mydataset)

train_size = int(len(mydataset) * 0.7)
test_size = len(mydataset) - train_size
train, test = mydataset[0:train_size,:], mydataset[train_size:len(mydataset),:]
print(len(train), len(test))

set options

In [0]:
batch_size = 8 
timestep = 16  
n_epochs = 100

create sequence

In [0]:
def create_sequence(dataset, window):
    dataX, dataY = [], []
    for i in range(len(dataset)-(window+1)):
        dataX.append(dataset[i:(i+window), 0])
        dataY.append(dataset[i + window, 0])
    
    return np.array(dataX), np.array(dataY)

# reshape into X=t and Y=t+1
trainX, trainY = create_sequence(train, timestep)
testX, testY = create_sequence(test, timestep)

reshape for lstm

In [0]:
trainX = np.reshape(trainX, (trainX.shape[0], timestep, 1))
testX = np.reshape(testX, (testX.shape[0], timestep, 1))

LSTM


In [0]:
model = Sequential()
model.add(LSTM(units=20, input_shape=(timestep, 1),return_sequences=False, stateful=False))
model.add(Dropout(0.2))
model.add(Dense(1)) 
model.compile(loss='mean_squared_error', optimizer='adam')

MLP

In [0]:
model = Sequential()
model.add(Dense(12, input_dim=timestep, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

fit

In [0]:
model.fit(trainX, trainY, batch_size=batch_size, epochs=n_epochs)

predict

In [0]:
trainPredict = model.predict(trainX, batch_size=batch_size)
testPredict = model.predict(testX, batch_size=1)

evaluate


In [0]:
# invert predictions
trainPredict = myscaler.inverse_transform(trainPredict)
trainY = myscaler.inverse_transform([trainY])
testPredict = myscaler.inverse_transform(testPredict)
testY = myscaler.inverse_transform([testY])

# evaluate
trainmse = mean_squared_error(trainY[0], trainPredict[:,0])
testmse = mean_squared_error(testY[0], testPredict[:,0])
trainmae = mean_absolute_error(trainY[0], trainPredict[:,0])
testmae = mean_absolute_error(testY[0], testPredict[:,0])

print('Train MSE: {0:.2f}'.format(trainmse))
print('Test MSE: {0:.2f}'.format(testmse))

print('Train MAE: {0:.2f}'.format(trainmae))
print('Test MAE: {0:.2f}'.format(testmae))

plot predictions

In [0]:
# shift train predictions for plotting
trainPredictshift = np.empty_like(mydataset)
trainPredictshift[:, :] = np.nan
trainPredictshift[timestep:len(trainPredict)+timestep, :] = trainPredict
# shift test predictions for plotting
testPredictshift = np.empty_like(mydataset)
testPredictshift[:, :] = np.nan
testPredictshift[len(trainPredict)+(timestep*2)+1:len(mydataset)-1, :] = testPredict

# plot predictions
plt.plot(mydataset, label='dataset')
plt.plot(trainPredictshift, label='train predictions')
plt.legend()
plt.show()

In [0]:
plt.plot(mydataset, label='dataset')
plt.plot(testPredictshift, label='test predictions')
plt.legend()
plt.show()

test different time steps

In [0]:
mses = []
time_steps = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32]
for time_step in time_steps:

    # create sequence
    trainX, trainY = create_sequence(train, timestep)
    testX, testY = create_sequence(test, timestep)

    # lstm
    trainX = np.reshape(trainX, (trainX.shape[0], timestep, 1))
    testX = np.reshape(testX, (testX.shape[0], timestep, 1))
    model = Sequential()
    model.add(LSTM(units=20, input_shape=(timestep, 1),return_sequences=False, stateful=False))
    model.add(Dropout(0.2))
    model.add(Dense(1)) 
    model.compile(loss='mean_squared_error', optimizer='adam')

    # Multilayer Perceptron
    # model = Sequential()
    # model.add(Dense(12, input_dim=timestep, activation='relu'))
    # model.add(Dense(8, activation='relu'))
    # model.add(Dense(1))
    # model.compile(loss='mean_squared_error', optimizer='adam')


    # fit
    model.fit(trainX, trainY, batch_size=batch_size, epochs=5)


    # predict
    trainPredict = model.predict(trainX, batch_size=batch_size)
    testPredict = model.predict(testX, batch_size=1)
    trainPredict = myscaler.inverse_transform(trainPredict)
    trainY = myscaler.inverse_transform([trainY])
    testPredict = myscaler.inverse_transform(testPredict)
    testY = myscaler.inverse_transform([testY])
    testmse = mean_squared_error(testY[0], testPredict[:,0])
    mses.append(testmse)
    print('Test MSE: {0:.2f}'.format(testmse))


print(mses)

plt.plot(np.array(time_steps), np.array(mses),label="mse's")
plt.legend()
